# Orbital-Optimization for reducing the one-norm

Author : Dr. Saad Yalouz - MSc Emiel Koridon (Cool titles)

A starting point to minimize the One-norm using Orbital-Optimization and a gradient free optimizer

## Theory 
The idea here is to employ a unitary transformation to modify the MOs of the system in order to minimize the One-norm. To build up this unitary, we start by the definition of the Orbital rotation operator which reads
$$ U(\vec{\kappa}) =\sum_{t>u}^{active} \kappa_{tu} (\hat{E}_{tu} - \hat{E}_{ut})$$
with $\hat{E}_{tu}=\frac{1} {\sqrt{2}}\sum_{\sigma}a^\dagger_{t,\sigma} a_{u,\sigma}$ and $\kappa_{tu}$ a rotation parameter affecting MOs $t$ and $u$ (note the absence of self rotation here with $t\neq u$). This operator takes the form of single excitations (with $t,u$ indices running only in the active space). Starting from this definition, a little bit of aglebra can show that implementing this rotation operator on creation/anihilation operators is strictly equivalent to redefining the MO coeff matrix of the Hamiltonian as follows
$$ \mathbf{C}_{new}(\vec{\kappa}) = \mathbf{C}_{ref} \mathbf{U}_{OO}(\vec{\kappa}) $$ 
Here, $\mathbf{U}_{OO}(\vec{\kappa})$ is a matrix representation of the effect of the rotations operator on the MOs. This matrix is an exponentiated version of the skew-matrix noted $\mathbf{K}$ encoding the parameter $\vec{\kappa}$. In other words
$$\mathbf{U}_{OO}(\vec{\kappa}) = e^{-\mathbf{K}} \text{,  with  } \mathbf{K} = skew(\vec{\kappa}) = \pmatrix{ 0 & \kappa_{01} & \kappa_{02} & \ldots & \kappa_{0N} \\
-\kappa_{01} & 0 & \kappa_{12} & \ldots & \kappa_{1N} \\
-\kappa_{02} & -\kappa_{12} & 0 &\ldots & \vdots \\
\vdots & \vdots & \vdots &\ddots & \vdots \\
-\kappa_{0N} & -\kappa_{1N} & \ldots & \ldots & 0 }$$

## Numerical implementation

Starting from this, we can encode this exponential numerically and use the kappa parameters (total number of parameters = $N(N-1)/2$) as degrees of freedom to reduce the one-norm. For this, first I use scimpy symbols to encode the matrix.

In [18]:
import psi4
import sys
import os
import openfermion
from openfermionpyscf import run_pyscf, compute_integrals
import numpy as np
import copy
from scipy.optimize import minimize
import math as m
import scipy 
import sympy
from sympy.matrices import zeros
from random import random
from openfermion import ops, InteractionOperator, get_sparse_operator, jordan_wigner
from openfermion.functionals import JW1norm_spat
from pyscf import gto, scf, lo, tools, fci, ao2mo, mcscf, df
import module as md
import time

# Set molecule parameters.
basis = ['sto-3g', 'cc-pvdz','def2svp'][0]
multiplicity = 1

# Set calculation parameters.
consider_cas = 1 # Do we consider an active space or the full space?
# Set size of active space
n_orbitals = 8 
n_electrons = 8
# Choose whether you want to start from localized orbitals or canonical orbitals
localize = 0
# Import .xyz geometry. This is kind of an involved code because I have many different geometries available...
# If you want to consider a Hydrogen chain or ring, set the following to 1
H_chain = 0
H_ring = 0
run_fci = 0
run_casci = 0
if H_chain:
    N_atoms = 14
    r = 1.8
    if consider_cas:
        description = 'H' + str(N_atoms) + str(basis) + 'ne' + str(n_electrons) +\
            'no' + str(n_orbitals)
    else:
        description = 'H' + str(N_atoms) + str(basis)
    geometry = [('H',( 0., 0., z*r)) for z in range(N_atoms)]
    
elif H_ring:
    theta = 88 * np.pi/180
    r = 1.3
    # Introduction of the molecular structure (.txt file)
    geometry = [
                ('H', (r*np.cos(theta/2.),   r*np.sin(theta/2.),  0.)),
                ('H', (r*np.cos(theta/2.),   -r*np.sin(theta/2.), 0.)),
                ('H', (-r*np.cos(theta/2.),  r*np.sin(theta/2.),  0.)),
                ('H', (-r*np.cos(theta/2.),  -r*np.sin(theta/2.), 0.))
                ]
    if consider_cas:
        description = 'H4' + str(basis) + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = 'H4' + str(basis)

else:
    fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
             'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
             'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
             'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
             'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
             'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt'][8]
    geometry = md.xyz_to_geom(fname)
    if consider_cas:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
            + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

# Determine number of AOs and electrons
nao = md.count_ao(geometry,basis,spin=multiplicity-1)
nelec = md.count_elec(geometry,basis,spin=multiplicity-1)
nmo = nao
print('nao:',nao,'\nnelec:', nelec)
threshold = 1e-10

# Set active indices
if consider_cas:
    ncore = (nelec - n_electrons) // 2
    ntot = ncore + n_orbitals # Number of core orbitals + number of active orbitals
    active_indices = list(range(ncore,ntot))
    occupied_indices = list(range(ncore))
else:
    ncore = 0
    ntot = nmo
    active_indices = list(range(nmo))

# Build molecule and run RHF
mol = gto.Mole()
mol.atom = geometry
mol.spin = multiplicity - 1
mol.symmetry = 0
mol.build()

if multiplicity == 1:
    myhf = scf.RHF(mol)
else:
    myhf = scf.ROHF(mol)

myhf.run()

# Extract MO_coeff and integrals. Then determine qubit 1-norm for CMOs and Pipek-Mezey MOs.

C_nonloc = np.copy(myhf.mo_coeff)

constant = float(mol.energy_nuc())


print('---------NON-LOCALIZED_ORBITALS---------')

C_copy = np.copy(C_nonloc)
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_copy, threshold)
if consider_cas:
    CASconstant, one_body_integrals, two_body_integrals =\
    md.get_active_space_integrals(one_body_integrals,
                                  two_body_integrals,
                                  occupied_indices,
                                  active_indices)
else:
    CASconstant = 0
t4 = time.time()
qub1norm = JW1norm_spat(np.copy(constant+CASconstant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('\ncalculating norm of qubit hamiltonian took', time.time()-t4)

print('\n')

print('---------LOCALIZED_ORBITALS---------')
localizemethod = 'Pipek-Mezey'

C = np.copy(C_nonloc)
if localizemethod == 'Pipek-Mezey':
    orb = lo.PipekMezey(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'Boys':
    orb = lo.Boys(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'ibo':
    orb = lo.ibo.ibo(mol, C[:,ncore:ntot])
elif localizemethod == 'ER':
    orb = lo.EdmistonRuedenberg(mol).kernel(C[:,ncore:ntot])
else:
    raise ValueError('Localization method not recognized')

C_locPM = np.hstack((C_nonloc[:,:ncore],orb,C_nonloc[:,ntot:nmo]))
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM, threshold)
if consider_cas:
    CASconstant, one_body_integrals, two_body_integrals =\
    md.get_active_space_integrals(one_body_integrals,
                                  two_body_integrals,
                                  occupied_indices,
                                  active_indices)
else:
    CASconstant = 0
t4 = time.time()
qub1norm_locPM = JW1norm_spat(np.copy(constant+CASconstant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('calculating norm of qubit hamiltonian took', time.time()-t4)

if consider_cas:
    print("Considering 1-norm of active space Hamiltonian")
else:
    print("Considering 1-norm of full space Hamiltonian")
    
print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)


OPT_OO_MAX_ITER  = 50000 # Maximum number of steps for the OO

# Building the K matrix : generator for the MO rotations 
Rot_param_values = []
Rot_param_names  = []
bounds = (-1.,1.)

K_mat_sym = zeros( nmo )
bounds = []
for q in range(nmo-1):
    for p in range(q+1,nmo):
        if ( p in active_indices and q in active_indices ):
            K_mat_sym[ p, q ] = - sympy.Symbol( "K_{}{}".format( p, q ) )
            K_mat_sym[ q, p ] = - K_mat_sym[ p, q ] 
            Rot_param_names  += [sympy.Symbol( "K_{}{}".format( p, q ))]
            Rot_param_values += [0 * (random()-0.5)] # <== Set starting Rot_param_values here.
            bounds += [ [-1., 1.] ] 

# Building the bounds for the rotational parameter amplitudes in the form of constraints
# (simply beacause cobyla doesn't have intrinsic bound options)
cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)


nao: 13 
nelec: 16
converged SCF energy = -92.8222185440788
---------NON-LOCALIZED_ORBITALS---------

calculating norm of qubit hamiltonian took 0.006116151809692383


---------LOCALIZED_ORBITALS---------
calculating norm of qubit hamiltonian took 0.0038177967071533203
Considering 1-norm of active space Hamiltonian
starting 1-norm nonloc is: 119.74890930870792
1norm locPM is: 113.89633221029666


In [19]:
K      = np.array( K_mat_sym.subs(list(zip(Rot_param_names,Rot_param_values))) ).astype(np.float64)  
U_OO   = scipy.linalg.expm( - K )

print("check if U_OO is unitary:",np.allclose(np.eye(U_OO.shape[0]), U_OO @ U_OO.T))

check if U_OO is unitary: True


Then I define a cost function to encode the evaluation of the One-Norm using the new MO coeff matrix. In the latter, we compute the 1-norm from the integrals. To proceed, we use $\mathbf{C}_{new}(\vec{\kappa})$ to build up the new electronic integrals.

In [20]:
def K_matr(Rot_param_values, active_indices):
    K = zeros( nmo )
    n = 0
    for q in range(nmo-1):
        for p in range(q+1,nmo):
            if ( p in active_indices and q in active_indices ):
                K[ p, q ] = - Rot_param_values[n]
                K[ q, p ] = - K[ p, q ]
                n += 1
    return np.array(K)

def Cost_function_OO_OneNorm(Rot_param_values):
#                              Rot_param_names,
#                              K_matr):
    """
    Cost function to minimize the One-Norm using MO rotations.
    """
    t6 = time.time()
    t5 = time.time()
    K = K_matr(Rot_param_values, active_indices)
    print("\nInitializing K-matrix took:",time.time()-t5)
    t1 = time.time()
    U_OO   = scipy.linalg.expm( - K )
    print("exponentiating matrix took:",time.time()-t1)
    t3 = time.time()
    if localize:
        C_MO   = C_locPM @ U_OO
    else:
        C_MO   = C_nonloc @ U_OO
    print("Matrix multiplication took:",time.time()-t3)
    t2 = time.time()
    one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_MO, threshold)
    print("extracting integrals took:",time.time()-t2)
    t4 = time.time()
    if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
    else:
        CASconstant = 0
    t4 = time.time()
    OneNorm = JW1norm_spat(np.copy(constant+CASconstant),
                           np.copy(one_body_integrals),
                           np.copy(two_body_integrals))
    print('calculating norm of qubit hamiltonian took', time.time()-t4)
    print('1-norm is:',OneNorm)
    print("total time for 1 cost function evaluation is", time.time()-t6)
    return OneNorm

print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)
t7 = time.time()
f_min_OO = minimize( Cost_function_OO_OneNorm,
                      x0      = Rot_param_values,
#                       args    = (Rot_param_names,
#                                  K_mat_sym),
                      constraints=cons,
                      method  = 'cobyla',
                      options = {'maxiter': OPT_OO_MAX_ITER,
                                'tol'    : 1e-5,
                                'disp': True}  )
print("total time for minimization with",OPT_OO_MAX_ITER,"max iterations was:", time.time()-t7)

starting 1-norm nonloc is: 119.74890930870792
1norm locPM is: 113.89633221029666

Initializing K-matrix took: 0.0019707679748535156
exponentiating matrix took: 0.0005123615264892578
Matrix multiplication took: 1.33514404296875e-05
extracting integrals took: 0.027634620666503906
calculating norm of qubit hamiltonian took 0.004247903823852539
1-norm is: 119.74890930870792
total time for 1 cost function evaluation is 0.035572052001953125

Initializing K-matrix took: 0.002414226531982422
exponentiating matrix took: 0.0006070137023925781
Matrix multiplication took: 8.821487426757812e-06
extracting integrals took: 0.015473365783691406
calculating norm of qubit hamiltonian took 0.003698587417602539
1-norm is: 119.31555698506806
total time for 1 cost function evaluation is 0.02304673194885254

Initializing K-matrix took: 0.0024132728576660156
exponentiating matrix took: 0.0005819797515869141
Matrix multiplication took: 8.344650268554688e-06
extracting integrals took: 0.01908421516418457
calcul

exponentiating matrix took: 0.0008604526519775391
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.019479751586914062
calculating norm of qubit hamiltonian took 0.005612373352050781
1-norm is: 117.20406029458765
total time for 1 cost function evaluation is 0.029597997665405273

Initializing K-matrix took: 0.002943277359008789
exponentiating matrix took: 0.0009717941284179688
Matrix multiplication took: 1.0251998901367188e-05
extracting integrals took: 0.017108440399169922
calculating norm of qubit hamiltonian took 0.003925800323486328
1-norm is: 125.475466961776
total time for 1 cost function evaluation is 0.02625751495361328

Initializing K-matrix took: 0.0026192665100097656
exponentiating matrix took: 0.0012388229370117188
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.01961803436279297
calculating norm of qubit hamiltonian took 0.004910469055175781
1-norm is: 117.81605368771199
total time for 1 cost function evaluation is 0

exponentiating matrix took: 0.0012722015380859375
Matrix multiplication took: 1.0728836059570312e-05
extracting integrals took: 0.03317403793334961
calculating norm of qubit hamiltonian took 0.006584882736206055
1-norm is: 119.02824326278962
total time for 1 cost function evaluation is 0.04769706726074219

Initializing K-matrix took: 0.003144979476928711
exponentiating matrix took: 0.0007371902465820312
Matrix multiplication took: 8.821487426757812e-06
extracting integrals took: 0.03653240203857422
calculating norm of qubit hamiltonian took 0.0029964447021484375
1-norm is: 122.34687439827147
total time for 1 cost function evaluation is 0.04433631896972656

Initializing K-matrix took: 0.002450227737426758
exponentiating matrix took: 0.0007116794586181641
Matrix multiplication took: 9.059906005859375e-06
extracting integrals took: 0.024245738983154297
calculating norm of qubit hamiltonian took 0.0038194656372070312
1-norm is: 119.84020092855577
total time for 1 cost function evaluation i

exponentiating matrix took: 0.0012912750244140625
Matrix multiplication took: 9.059906005859375e-06
extracting integrals took: 0.016704797744750977
calculating norm of qubit hamiltonian took 0.003751516342163086
1-norm is: 118.31568762063404
total time for 1 cost function evaluation is 0.02547454833984375

Initializing K-matrix took: 0.0030248165130615234
exponentiating matrix took: 0.001112222671508789
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.017113208770751953
calculating norm of qubit hamiltonian took 0.0037703514099121094
1-norm is: 118.33862150081492
total time for 1 cost function evaluation is 0.025890111923217773

Initializing K-matrix took: 0.0028755664825439453
exponentiating matrix took: 0.0008273124694824219
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.015766382217407227
calculating norm of qubit hamiltonian took 0.004423379898071289
1-norm is: 119.27723371072562
total time for 1 cost function evaluation

extracting integrals took: 0.017623424530029297
calculating norm of qubit hamiltonian took 0.0035936832427978516
1-norm is: 118.0299955621943
total time for 1 cost function evaluation is 0.025982379913330078

Initializing K-matrix took: 0.002851724624633789
exponentiating matrix took: 0.0007319450378417969
Matrix multiplication took: 1.0251998901367188e-05
extracting integrals took: 0.017290830612182617
calculating norm of qubit hamiltonian took 0.0044400691986083984
1-norm is: 117.78993503355966
total time for 1 cost function evaluation is 0.02641463279724121

Initializing K-matrix took: 0.00483393669128418
exponentiating matrix took: 0.0010528564453125
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.016821622848510742
calculating norm of qubit hamiltonian took 0.005026102066040039
1-norm is: 117.89191402297105
total time for 1 cost function evaluation is 0.028945446014404297

Initializing K-matrix took: 0.0031130313873291016
exponentiating matrix took: 0.

...

exponentiating matrix took: 0.001645803451538086
Matrix multiplication took: 8.58306884765625e-06
extracting integrals took: 0.01893305778503418
calculating norm of qubit hamiltonian took 0.005773067474365234
1-norm is: 116.02433499642731
total time for 1 cost function evaluation is 0.030565738677978516

Initializing K-matrix took: 0.002886056900024414
exponentiating matrix took: 0.001252889633178711
Matrix multiplication took: 1.0728836059570312e-05
extracting integrals took: 0.014050006866455078
calculating norm of qubit hamiltonian took 0.005903005599975586
1-norm is: 116.02425310492482
total time for 1 cost function evaluation is 0.02511119842529297

Initializing K-matrix took: 0.003919124603271484
exponentiating matrix took: 0.0007901191711425781
Matrix multiplication took: 8.106231689453125e-06
extracting integrals took: 0.016613483428955078
calculating norm of qubit hamiltonian took 0.0036361217498779297
1-norm is: 116.02418857583712
total time for 1 cost function evaluation is 

extracting integrals took: 0.0187222957611084
calculating norm of qubit hamiltonian took 0.005358219146728516
1-norm is: 116.02417954944808
total time for 1 cost function evaluation is 0.02915167808532715

Initializing K-matrix took: 0.0039424896240234375
exponentiating matrix took: 0.0008904933929443359
Matrix multiplication took: 1.0013580322265625e-05
extracting integrals took: 0.019039392471313477
calculating norm of qubit hamiltonian took 0.006024837493896484
1-norm is: 116.02405051463012
total time for 1 cost function evaluation is 0.03109884262084961

Initializing K-matrix took: 0.002905130386352539
exponentiating matrix took: 0.0010454654693603516
Matrix multiplication took: 9.059906005859375e-06
extracting integrals took: 0.02430415153503418
calculating norm of qubit hamiltonian took 0.0038030147552490234
1-norm is: 116.024138473712
total time for 1 cost function evaluation is 0.03278183937072754

Initializing K-matrix took: 0.0030100345611572266
exponentiating matrix took: 0.

Initializing K-matrix took: 0.0028083324432373047
exponentiating matrix took: 0.0007100105285644531
Matrix multiplication took: 8.106231689453125e-06
extracting integrals took: 0.01866912841796875
calculating norm of qubit hamiltonian took 0.006017923355102539
1-norm is: 116.02379841549022
total time for 1 cost function evaluation is 0.0324397087097168

Initializing K-matrix took: 0.0029735565185546875
exponentiating matrix took: 0.0008146762847900391
Matrix multiplication took: 9.775161743164062e-06
extracting integrals took: 0.01717543601989746
calculating norm of qubit hamiltonian took 0.00665593147277832
1-norm is: 116.02381539187202
total time for 1 cost function evaluation is 0.028975963592529297

Initializing K-matrix took: 0.0035102367401123047
exponentiating matrix took: 0.0010797977447509766
Matrix multiplication took: 9.298324584960938e-06
extracting integrals took: 0.01532292366027832
calculating norm of qubit hamiltonian took 0.0037207603454589844
1-norm is: 116.0237735339

calculating norm of qubit hamiltonian took 0.0045125484466552734
1-norm is: 116.0236837983888
total time for 1 cost function evaluation is 0.025556087493896484

Initializing K-matrix took: 0.005026340484619141
exponentiating matrix took: 0.0019483566284179688
Matrix multiplication took: 8.821487426757812e-06
extracting integrals took: 0.024352073669433594
calculating norm of qubit hamiltonian took 0.006494998931884766
1-norm is: 116.02367423478003
total time for 1 cost function evaluation is 0.03904366493225098

Initializing K-matrix took: 0.002885103225708008
exponentiating matrix took: 0.001096487045288086
Matrix multiplication took: 1.0013580322265625e-05
extracting integrals took: 0.015761375427246094
calculating norm of qubit hamiltonian took 0.0044553279876708984
1-norm is: 116.02363732158703
total time for 1 cost function evaluation is 0.025310277938842773

Initializing K-matrix took: 0.002917051315307617
exponentiating matrix took: 0.001146554946899414
Matrix multiplication too

extracting integrals took: 0.016077041625976562
calculating norm of qubit hamiltonian took 0.0038356781005859375
1-norm is: 116.02350385527797
total time for 1 cost function evaluation is 0.024770498275756836

Initializing K-matrix took: 0.00421595573425293
exponentiating matrix took: 0.0010979175567626953
Matrix multiplication took: 9.5367431640625e-06
extracting integrals took: 0.020978927612304688
calculating norm of qubit hamiltonian took 0.005457162857055664
1-norm is: 116.02350169259238
total time for 1 cost function evaluation is 0.032686471939086914

Initializing K-matrix took: 0.002887248992919922
exponentiating matrix took: 0.0010700225830078125
Matrix multiplication took: 9.298324584960938e-06
extracting integrals took: 0.015308618545532227
calculating norm of qubit hamiltonian took 0.004346132278442383
1-norm is: 116.02350110871615
total time for 1 cost function evaluation is 0.024349451065063477

Initializing K-matrix took: 0.003539562225341797
exponentiating matrix took: 

In [21]:
print("message:",f_min_OO.message,"number of function evaluations:",f_min_OO.nfev)

message: Optimization terminated successfully. number of function evaluations: 1217


In [22]:
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_OO, threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorbOO = JW1norm_spat(np.copy(constant+CASconstant),
                            np.copy(one_body_integrals),
                            np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_nonloc, threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorbnonloc = JW1norm_spat(np.copy(constant+CASconstant),
                                np.copy(one_body_integrals),
                                np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM, threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorblocPM = JW1norm_spat(np.copy(constant+CASconstant),
                       np.copy(one_body_integrals),
                       np.copy(two_body_integrals))
if consider_cas:
    print("Considering CAS(" + str(n_electrons) + "," + str(n_orbitals) + "), \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)
else:
    print("Considering full space, \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)

Considering CAS(8,8), 
1norm of CMO orb is 119.74890930870792 
1norm of PM orb is 113.89633221029666 
1norm of optimizer is 116.0234404541888


## Visualize
If you want to save the orbitals to gaussian CUBE files

In [ ]:
Cost_function_OO_OneNorm(f_min_OO.x)
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO
t13 = time.time()
for i in range(nmo):    
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + 'onenorm_orb' + localizemethod + str(localize)\
                          + str(consider_cas) + str(i) , C_OO[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)